In [1]:
def tensorflow_model_local_absolute(mr, name, version=None, metrics=None, description=None, input_example=None, model_schema=None):
    print("LOCAL ABSOLUTE MODEL PATH")
    import os
    import random
    from hsml import model
    
    model_path = os.getcwd() + "/model"
    if not os.path.exists(model_path):
        os.mkdir(model_path)
        f = open(model_path + "/saved_model.pb", "w+")
        f.write("model")
        f.close()
    
    tf_model = mr.tensorflow.create_model(name, metrics=metrics, description=description, input_example=input_example, model_schema=model_schema)
    ret = tf_model.save(model_path)
    return ret

Starting Spark application


SparkSession available as 'spark'.


In [2]:
def tensorflow_model_local_relative(mr, name, version=None, metrics=None, description=None, input_example=None, model_schema=None):
    print("LOCAL RELATIVE MODEL PATH")
    import os
    import random
    from hsml import model
    
    model_path = os.getcwd() + "/model"
    if not os.path.exists(model_path):
        os.mkdir(model_path)
        f = open(model_path + "/saved_model.pb", "w+")
        f.write("model")
        f.close()
    
    tf_model = mr.tensorflow.create_model(name, metrics=metrics, description=description, input_example=input_example, model_schema=model_schema)
    ret = tf_model.save('model')
    return ret

In [3]:
def sklearn_model(mr, name, metrics=None, description=None, input_example=None, model_schema=None):
    
    print("HDFS ABSOLUTE WITH HDFS PREFIX MODEL PATH")
    
    import os
    import random
    from hsml import model
    from hops import hdfs
    
    # Export model from project absolute hdfs path with hdfs prefix
    model_path = "hdfs:///Projects/{}/Resources".format(hdfs.project_name())

    sklearn_model = mr.sklearn.create_model(name, metrics=metrics, description=description, input_example=input_example, model_schema=model_schema)
    ret = sklearn_model.save(model_path)
    return ret

In [4]:
def torch_model(mr, name, metrics=None, description=None, input_example=None, model_schema=None):
    
    print("HDFS ABSOLUTE WITHOUT HDFS PREFIX MODEL PATH")
    
    import os
    import random
    from hsml import model
    from hops import hdfs
    
    # Export model from project absolute hdfs path
    model_path = "/Projects/{}/Resources".format(hdfs.project_name())

    torch_model = mr.torch.create_model(name, metrics=metrics, description=description, input_example=input_example, model_schema=model_schema)
    ret = torch_model.save(model_path)
    return ret

In [5]:
def python_model(mr, name, metrics=None, description=None, input_example=None, model_schema=None):
    
    print("HDFS RELATIVE MODEL PATH")
    
    import os
    import random
    from hsml import model
    
    # Export model from project relative hdfs path
    model_path = "Resources"

    python_model = mr.python.create_model(name, metrics=metrics, description=description, input_example=input_example, model_schema=model_schema)

    ret = python_model.save(model_path)
    return ret

In [6]:
def export_tf():
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import numpy
    import pandas
    import os
    import hsml
    from hsml.client.exceptions import RestAPIError
    
    connection = hsml.connection()
    mr = connection.get_model_registry()
    
    # tf model 1
    metrics = {'accuracy': 0.333, 'loss': 0.4}

    description = "A test desc for this model"

    input_data = {'int_column': [1, 2], 'string_column': ["John", "Jamie"], 'float_column': [0.5, 0.3]}
    input_df = pandas.DataFrame(data=input_data)

    predictions = numpy.array([1.1, 20.2, 30.3, 40.4, 50.0, 60.6, 70.7, 0.1])

    model_schema = ModelSchema(input_schema=Schema(input_df), output_schema=Schema(predictions))

    exported_tf_model = tensorflow_model_local_absolute(mr, "model_tf", version=1, metrics=metrics, description=description, model_schema=model_schema, input_example=input_df)

    # tf model 2
    metrics = {'accuracy': 0.95, 'loss': 0.4}

    exported_tf_model = tensorflow_model_local_absolute(mr, "model_tf", metrics=metrics, description=description, model_schema=model_schema, input_example=input_df)
    
    # tf model 3
    metrics = {'accuracy': 0.90, 'loss': 0.4}

    exported_tf_model = tensorflow_model_local_relative(mr, "model_tf", version=3, metrics=metrics, description=description, model_schema=model_schema, input_example=input_df)

In [7]:
def export_py():
    import numpy
    import pandas
    import os
    import hsml
    from hsml.client.exceptions import RestAPIError
    
    connection = hsml.connection()
    mr = connection.get_model_registry()
    
    input_data = {'int_column': [1, 2], 'string_column': ["John", "Jamie"], 'float_column': [0.5, 0.3]}
    input_df = pandas.DataFrame(data=input_data)
    
    exported_python_model = python_model(mr, "model_python", input_example=input_df)
    assert isinstance(exported_python_model, hsml.python.model.Model)

In [8]:
def export_sklearn():
    import numpy
    import pandas
    import os
    import hsml
    from hsml.client.exceptions import RestAPIError
    
    connection = hsml.connection()
    mr = connection.get_model_registry()
    
    input_data = {'int_column': [1, 2], 'string_column': ["John", "Jamie"], 'float_column': [0.5, 0.3]}
    input_df = pandas.DataFrame(data=input_data)
    
    exported_sklearn_model = sklearn_model(mr, "model_sklearn", input_example=input_df)
    assert isinstance(exported_sklearn_model, hsml.sklearn.model.Model)

In [9]:
def export_torch():
    import numpy
    import pandas
    import os
    import hsml
    from hsml.client.exceptions import RestAPIError
    
    connection = hsml.connection()
    mr = connection.get_model_registry()
    
    input_data = {'int_column': [1, 2], 'string_column': ["John", "Jamie"], 'float_column': [0.5, 0.3]}
    input_df = pandas.DataFrame(data=input_data)
    
    exported_torch_model = torch_model(mr, "model_torch", input_example=input_df)
    assert isinstance(exported_torch_model, hsml.torch.model.Model)

In [10]:
from hops import experiment

#Export from local paths
experiment.launch(export_tf)

#Export from hdfs paths
experiment.launch(export_py)
experiment.launch(export_sklearn)
experiment.launch(export_torch)

Finished Experiment 

Finished Experiment 

Finished Experiment 

Finished Experiment 

('hdfs://rpc.namenode.service.consul:8020/Projects/demo_ml_meb10000/Experiments/application_1637773802226_0043_4', {'metric': None, 'log': 'Experiments/application_1637773802226_0043_4/output.log'})